In [ ]:
### Install Required Dependencies (Run this cell first)
!pip -q install chromadb openai langchain langchain-community langchain-openai tiktoken pymupdf
!pip -q install google-generativeai langchain-community
!pip install -q langchain-google-genai
!pip -q install docx2txt unstructured

In [2]:
### Import Necessary Libraries
import os
from langchain.vectorstores import Chroma
from langchain.document_loaders import AsyncChromiumLoader #directlyt import the data from an URL
from langchain.document_transformers import Html2TextTransformer #converts html to text data
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

In [3]:
### Set API Key for Google Gemini
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = api_key

In [33]:
### Load and Process Documents
# Load the PDF file
# loader = PyMuPDFLoader("/content/FIlterign review.pdf")
# document = loader.load()

document = []
for filename in os.listdir("documents"):
    if filename.lower().endswith(".pdf"):
        loader = PyMuPDFLoader(os.path.join("documents", filename))
        document.extend(loader.load())

print(f"Total pages loaded: {len(document)}")

Total pages loaded: 2


In [29]:
# Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_chunks = text_splitter.split_documents(document)

len(text_chunks)

9

In [21]:
### Set Up Vector Database (ChromaDB)
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


# Create or load a vector database
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=text_chunks,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [22]:
retriever = vectordb.as_retriever()

In [23]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

In [24]:
### Function to Ask LLM First
def ask_llm_first(query):
    """Ask the LLM first. If it confidently answers, return it. Otherwise, proceed to VectorDB."""

    system_prompt = (
        "You are an AI assistant. Answer only if you are confident about your response. "
        "If you do not know the answer, simply respond with: 'I don't know.' "
        "Do not guess or make assumptions.\n\n"
        f"User Question: {query}"
    )

    response = llm.invoke(system_prompt)  # Query the LLM

    # Extract text content from AIMessage object
    if hasattr(response, "content"):
        response_text = response.content  # Correct way to get the response text
    else:
        response_text = str(response)  # Fallback in case 'content' is missing

    # Define phrases that indicate uncertainty
    vague_responses = [
        "i don't know", "i am not sure", "i need more information", "please provide more details",
        "i cannot determine", "i am unable to", "i do not have enough context",
        "i need more context", "i do not have enough information", "i require additional details"
    ]

     # Debugging: Print each phrase check
    for phrase in vague_responses:
        if phrase in response_text.lower():
            print(f"✅ Match found: '{phrase}' is in LLM response!")
            print("⚠️ LLM is unsure. Fetching data from VectorDB...")
            return False  # Indicate retrieval is needed

    print("🤖 LLM confidently answered without RAG.")
    return response_text  # Return LLM's answer

In [25]:
### Function to Perform RAG Retrieval
def ask_with_rag(query):
    """Retrieve documents from VectorDB and generate an answer using LLM with context."""
    retrieved_docs = retriever.invoke(query)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)
    response = qa_chain.invoke({"context": retrieved_docs, "query": query})

    print("\n🤖 AI Agent's Response (with RAG):")
    return response["result"]


In [26]:
def start_conversation():
    """Start an interactive chat session with the LLM and VectorDB."""
    print("\n💬 Start chatting with the AI! (Type 'exit' to stop)\n")

    while True:
        # Get user input dynamically
        user_query = input("👤 You: ").strip()

        # Exit condition
        if user_query.lower() == "exit":
            print("👋 Goodbye!")
            break

        # Step 1: Ask LLM first
        llm_response = ask_llm_first(user_query)

        # Step 2: If LLM is unsure (False), use VectorDB retrieval
        if llm_response is False:
            final_answer = ask_with_rag(user_query)
            print(f"\n🤖 AI: {final_answer}\n")
        else:
            print(f"\n🤖 AI: {llm_response}\n")


In [ ]:
start_conversation()